In [1]:
import rclpy
from rclpy.node import Node
import sys
import numpy as np
import collections
from pathlib import Path
import json
import shutil
from cookiecutter.main import cookiecutter
import math

rclpy.init(args=sys.argv)
node = rclpy.create_node('deneme')

1696870670.571400 [0]    python3: selected interface "lo" is not multicast-capable: disabling multicast


In [2]:
def get_param(name, default):
    try:
        value = node.declare_parameter(name, default).value
    except ConnectionRefusedError:
        value = default
    print('param "{}" = "{}"'.format(name, value))
    return value

SEED = int(get_param('~seed', '988'))
WORLD_NAME = get_param('~world_name', 'tomato_field')
MODEL_NAME_PREFIX = get_param('~model_name_prefix', 'tomato')
ROW_COUNT = int(get_param('~row_count', '3'))
ROW_LENGTH = int(get_param('~row_length', '3'))
ROW_DIST = float(get_param('~row_dist', '2.0'))
CROP_DIST = float(get_param('~crop_dist', '0.7'))
CROP_NUM_IN_A_ROW = int(math.floor(ROW_LENGTH/CROP_DIST + 1))
FARM_SIZE = [math.ceil((ROW_COUNT + 1) * ROW_DIST), math.ceil(ROW_LENGTH + 2 * ROW_DIST)]
FARM_NAME = str(FARM_SIZE[0]) + 'mx' + str(FARM_SIZE[1]) + 'm'

param "~seed" = "988"
param "~world_name" = "tomato_field"
param "~model_name_prefix" = "tomato"
param "~row_count" = "3"
param "~row_length" = "3"
param "~row_dist" = "2.0"
param "~crop_dist" = "0.7"


In [3]:
#OUT_PATH = Path(get_param('~out_path', str(Path().resolve().parent / 'generated/test01'))).resolve()
OUT_PATH = Path(get_param('~out_path', str(Path().resolve().parent / 'generated'))).resolve()
WORLD_TEMPLATE = Path(get_param('~world_template', str(Path().resolve().parent / 'templates/tomato_world_gazebo_classic_and_sim'))).resolve()
MODEL_TEMPLATE = Path(get_param('~model_template', str(Path().resolve().parent / 'templates/tomato_model_gazebo_classic_and_sim'))).resolve()

shutil.rmtree(OUT_PATH, ignore_errors=True)
np.random.seed(SEED)


param "~out_path" = "/home/ayilmaz/random_farm_ws/src/gazebo_tomato_farm_generator/generated"
param "~world_template" = "/home/ayilmaz/random_farm_ws/src/gazebo_tomato_farm_generator/templates/tomato_world_gazebo_classic_and_sim"
param "~model_template" = "/home/ayilmaz/random_farm_ws/src/gazebo_tomato_farm_generator/templates/tomato_model_gazebo_classic_and_sim"


In [7]:
# helper class to build the markers.json
class Markers:
    markers = []
    last_id = 0

    @staticmethod
    def next_id():
        Markers.last_id += 1
        return Markers.last_id

    @staticmethod
    def reset():
        Markers.markers = []
    
    @staticmethod
    def add_plant(x, y, z):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'PLANT',
            'id': id,
            'translation': [x, y, z]
        })
        return id
    

    @staticmethod
    def add_fruit(x, y, z, plant_id):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'FRUIT',
            'id': id,
            'translation': [x, y, z],
            'plant_id': plant_id
        })
        return id
    
    @staticmethod
    def dumps():
        return json.dumps(Markers.markers, indent=4)

In [9]:
models = {'list': []}
Markers.reset()

for x in range(ROW_COUNT):
    for y in range(CROP_NUM_IN_A_ROW):
        model_name = 'tomato_{}'.format(x * CROP_NUM_IN_A_ROW + y)

        cookiecutter(str(MODEL_TEMPLATE),
             output_dir=str(OUT_PATH) + '/' + str(FARM_NAME), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'model_name': model_name})

        x_pos, y_pos, z_pos = x * ROW_DIST - ((ROW_COUNT - 1)*ROW_DIST)/2, y * CROP_DIST - ROW_LENGTH/2, 0
        models['list'].append({
            'model': model_name,
            'name': model_name,
            'pose': '{} {} 0 0 0 0'.format(x_pos, y_pos)
        })
        x_pos += np.random.uniform(-0.1, 0.1)
        y_pos += np.random.uniform(-0.1, 0.1)
        seed = np.random.randint(10000)
        dir = (OUT_PATH / WORLD_NAME / FARM_NAME / model_name).resolve()
        dir_blender = (Path.cwd() / '../blender').resolve()
        blend = str(dir_blender / 'tomato_gen.blend')
        script = str(dir_blender / 'tomato_gen.py')
        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
        with open(dir / 'markers.json') as markers_file:
            plant_markers = json.load(markers_file)
            for marker in plant_markers:
                if marker['marker_type'] == 'FRUIT':
                    Markers.add_fruit(
                        marker['translation'][0] + x_pos,
                        marker['translation'][1] + y_pos,
                        marker['translation'][2] + z_pos,
                        plant_id
                    )

UndefinedVariableInTemplate: Unable to create project directory '{{cookiecutter.farm_name}}'. Error message: 'collections.OrderedDict object' has no attribute 'farm_name'. Context: OrderedDict([('cookiecutter', OrderedDict([('model_name', 'tomato_0'), ('world_name', 'tomato_field'), ('_template', '/home/ayilmaz/random_farm_ws/src/gazebo_tomato_farm_generator/templates/tomato_model_gazebo_classic_and_sim'), ('_output_dir', '/home/ayilmaz/random_farm_ws/src/gazebo_tomato_farm_generator/generated/8mx7m'), ('_repo_dir', '/home/ayilmaz/random_farm_ws/src/gazebo_tomato_farm_generator/templates/tomato_model_gazebo_classic_and_sim'), ('_checkout', None)])), ('_cookiecutter', {'model_name': 'tomato_0', 'world_name': 'tomato_field'})])

In [10]:
cookiecutter(str(WORLD_TEMPLATE),
             output_dir=str(OUT_PATH)+'/'+str(FARM_NAME), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'models': models, 'farm_area_x': str(FARM_SIZE[0]), 'farm_area_y': str(FARM_SIZE[1])})


'/home/ayilmaz/random_farm_ws/src/gazebo_tomato_farm_generator/generated/8mx7m/tomato_field'

In [ ]:
with open(OUT_PATH / WORLD_NAME / FARM_NAME / 'markers.json', 'w') as outfile:
    json.dump(Markers.markers, outfile, indent=4, sort_keys=True)